In [ ]:
#Code inspiration-
#https://github.com/rs75/bitcoin-price-model/blob/main/bitcoin_price_prediction.ipynb
#https://www.kaggle.com/marsescobin/predicting-bitcoin-prices-using-random-forest
!pip install --upgrade quandl
!pip install --upgrade ta



  Created wheel for ta: filename=ta-0.7.0-cp37-none-any.whl size=28716 sha256=aa3083dcce11c5c6fdeec04ab68b5ec17cb0e378d983f953d88ca5449273eb50
  Stored in directory: /root/.cache/pip/wheels/dd/88/30/de9553fb54a474eb7480b937cdbb140bdda613d29cf4da7994
Successfully built ta


In [ ]:
import quandl
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

df = quandl.get('BCHARTS/KRAKENUSD', returns="pandas")
df.head()

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-07,874.67040,892.06753,810.00000,810.00000,15.622378,13151.472844,841.835522
2014-01-08,810.00000,899.84281,788.00000,824.98287,19.182756,16097.329584,839.156269
2014-01-09,825.56345,870.00000,807.42084,841.86934,8.158335,6784.249982,831.572913
2014-01-10,839.99000,857.34056,817.00000,857.33056,8.024510,6780.220188,844.938794
2014-01-11,858.20000,918.05471,857.16554,899.84105,18.748285,16698.566929,890.671709


We could already try to use this dataframe to train a classification model, but I decided to add more features what usually makes a model more accurate.

I found a library called ta, which creates many technical indicators, like moving averages or bollinger bands (and many more). It is originally made for analyzing stock data, but should also work for bitcoin. The following code adds more than 80 features to the dataset. Check out the notebook on Github to see the output.

https://github.com/bukosabino/ta

In [ ]:
from ta import add_all_ta_features
from ta.utils import dropna

pd.set_option('display.max_columns', None)

df = dropna(df)
df = add_all_ta_features(
    df, open="Open", high="High", low="Low", close="Close", volume="Volume (BTC)")

df = df.fillna(0)
df.tail()

/usr/local/lib/python3.7/dist-packages/ta/trend.py:768: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i] / self._trs[i])
/usr/local/lib/python3.7/dist-packages/ta/trend.py:772: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i] / self._trs[i])


,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price,volume_adi,volume_obv,volume_cmf,volume_fi,volume_mfi,volume_em,volume_sma_em,volume_vpt,volume_nvi,volume_vwap,volatility_atr,volatility_bbm,volatility_bbh,volatility_bbl,volatility_bbw,volatility_bbp,volatility_bbhi,volatility_bbli,volatility_kcc,volatility_kch,volatility_kcl,volatility_kcw,volatility_kcp,volatility_kchi,volatility_kcli,volatility_dcl,volatility_dch,volatility_dcm,volatility_dcw,volatility_dcp,volatility_ui,trend_macd,trend_macd_signal,trend_macd_diff,trend_sma_fast,trend_sma_slow,trend_ema_fast,trend_ema_slow,trend_adx,trend_adx_pos,trend_adx_neg,trend_vortex_ind_pos,trend_vortex_ind_neg,trend_vortex_ind_diff,trend_trix,trend_mass_index,trend_cci,trend_dpo,trend_kst,trend_kst_sig,trend_kst_diff,trend_ichimoku_conv,trend_ichimoku_base,trend_ichimoku_a,trend_ichimoku_b,trend_visual_ichimoku_a,trend_visual_ichimoku_b,trend_aroon_up,trend_aroon_down,trend_aroon_ind,trend_psar_up,trend_psar_down,trend_psar_up_indicator,trend_psar_down_indicator,trend_stc,momentum_rsi,momentum_stoch_rsi,momentum_stoch_rsi_k,momentum_stoch_rsi_d,momentum_tsi,momentum_uo,momentum_stoch,momentum_stoch_signal,momentum_wr,momentum_ao,momentum_kama,momentum_roc,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,others_dr,others_dlr,others_cr
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021-05-30,34631.9,36491.0,33423.2,35673.8,4455.063045,1.572268e+08,35291.702478,1.036920e+06,508767.420695,-0.030986,-9.356634e+06,30.078292,-3.575951e+10,-3.882258e+10,-38.776482,41722.420398,38009.201062,4629.817183,41669.855,53842.667292,29497.042708,58.425028,0.253711,0.0,0.0,37044.030000,41597.130000,32490.930000,24.582099,0.349528,0.0,0.0,29800.0,57983.6,43891.80,67.635465,0.208412,29.255444,-4607.383775,-4437.266905,-170.116870,37322.683333,45301.280769,38546.871671,43154.255446,55.245174,5.711833,31.458610,0.861162,1.178038,-0.316875,-1.319465,26.544773,-79.951265,-4927.555,-302.471083,-270.382359,-32.088724,35986.15,44699.4,40342.775,47350.0,54927.50,55950.0,12.0,96.0,-84.0,30661.216580,0.0,0.0,0.0,0.000252,34.435185,0.780624,0.687488,0.742831,-32.544460,50.069701,34.862451,28.964339,-65.137549,-10741.052353,38964.324274,-16.834984,-0.255312,11.633758,-11.889070,3.054026,3.008319,4304.172840
2021-05-31,35667.9,37512.8,34200.0,37303.8,6224.605942,2.256991e+08,36259.176987,1.042360e+06,514992.026637,-0.019795,-6.570528e+06,35.308145,4.786168e+10,-2.834767e+10,420.472215,41722.420398,37505.562658,4498.115465,40697.670,50833.852512,30561.487488,49.812102,0.332586,0.0,0.0,36906.710000,40917.420000,32896.000000,21.734313,0.549504,0.0,0.0,29800.0,57983.6,43891.80,69.251139,0.266247,28.622795,-4365.258500,-4422.865224,57.606724,37369.475000,44524.615385,38355.629875,42720.888376,55.717371,7.026064,29.813552,0.892644,1.123973,-0.231329,-1.340262,26.359290,-62.535481,-83.070,-298.852009,-280.718521,-18.133488,35986.15,44699.4,40342.775,47350.0,55817.90,55950.0,8.0,92.0,-84.0,30865.864249,0.0,0.0,0.0,0.000126,38.467508,1.000000,0.792796,0.731840,-31.223459,50.773125,46.830261,35.247027,-53.169739,-10896.111176,38957.181405,1.528211,-2.637734,8.779459,-11.417193,4.569180,4.467867,4505.407407
2021-06-01,37273.1,37934.3,35748.0,36676.3,5703.032935,2.092728e+08,36695.002361,1.041500e+06,509288.993702,0.008956,-6.143117e+06,39.766483,3.775112e+10,-2.462127e+10,188.480755,41020.593270,37167.561922,4266.933919,40056.350,49481.083588,30631.616412,47.057376,0.320682,0.0,0.0,36866.656667,40736.846667,32996.466667,20.995611,0.475407,0.0,0.0,29800.0,51540.0,40670.00,54.273542,0.316297,27.802556,-4175.869656,-4373.466111,197.596455,37041.283333,43765.200000,38097.271433,42273.141089,55.935269,7.465139,28.745219,0.909472,1.109038,-0.199566,-1.350435,26.142244,-53.014442,-2724.650,-292.529718,-287.249983,-5.279735,37158.40,44699.4,40928.900,47350.0,55817.90,55950.0,4.0,88.0,-84.0,31066.418964,0.0,0.0,0.0,0.000063,37.511074,0.938425,0.906350,0.795544,-30.3

Applying Random Forest
To perform a Random Forest training on our dataset, we assign the X and y variables. X will have all the columns that we are assuming to be factors affecting price, y.

In [ ]:
interest = ["Close","Volume (BTC)", "trend_ema_fast", "momentum_rsi", "others_dlr"]
df2 = df[interest]
df2.describe()




features = ["Volume (BTC)",  "trend_ema_fast", "momentum_rsi", "others_dlr"]
X = df2[features]
y = df2['Close']

In [ ]:
#Then we split the dataset into a training and testing or validation dataset. We need this so we can test how good our predictions are to the actual prices.

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

We declare that we’re using Random Forest to create a model:

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state=1)

We now feed the training data to the Random Forest algorithm so it can map patterns or decision trees.

In [ ]:
rf_model.fit(train_X, train_y)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

Once we have a model, we can call predict to predict the prices of bitcoin on our testing or validation data. That is, given that we’ve trained a model using train_X and train_y, we are now interested if it’d be able to ouput a number close to val_Y given val_X. Remember that we split the dataset earlier to a training dataset (train_X, train_Y) and a testing dataset (val_X, val_y).

In [ ]:
rf_pred = rf_model.predict(val_X)

To see the predicted values of the model on the last 5 validation data, we can call:

In [ ]:
print(rf_model.predict(X.tail()))


[36298.966 37083.375 36469.865 37095.704 36944.116]


In [ ]:
df2['Close'].tail()

Date
2021-05-30    35673.8
2021-05-31    37303.8
2021-06-01    36676.3
2021-06-02    37565.3
2021-06-03    38816.2
Name: Close, dtype: float64

Let’s see how far off the predicted values are from the actual values using MAE (Mean Absolute Error). This number will tell us how off are predictions are on average

In [ ]:
from sklearn.metrics import mean_absolute_error
rf_val_mae = mean_absolute_error(val_y,rf_pred)
rf_val_mae

173.17544457910367

Not bad! MAE is telling us that on average, our bitcoin price prediction is off by $141.31 from the actual prices.

In [ ]:
 
#plt.plot(val_y, label="Actual Price", color='green')
#plt.plot(rf_pred, label="Predicted Price", color='red')

#plt.title('Bitcoin price prediction')
#plt.xlabel('Time [days]')
#plt.ylabel('Price')
#plt.legend(loc='best')

In [ ]:
from sklearn.metrics import r2_score
r2_score(val_y, rf_pred)

0.9985596214560466

In [ ]:
rmse = np.sqrt(((rf_pred - val_y) ** 2).mean())
print('Mean Squared Error: {}'.format(round(rmse, 2)))

Mean Squared Error: 381.94


In [ ]:
plt.plot(abs(val_y), label="Actual Price", color='green')
plt.plot(abs(rf_pred), label="Predicted Price", color='red')


plt.title('Bitcoin price prediction')
plt.xlabel('Time [days]')
plt.ylabel('Price')
plt.legend(loc='best')
 
plt.show();

ValueError: ignored